In [2]:
import sys
sys.path.append('../')

In [3]:
import warnings
import logging
import time
import psutil
import os

import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime
from multiprocessing import Manager
from multiprocessing.managers import BaseManager


from its_logging.logger_config import logger
from utils.its_utils import clip_to_california, get_wfr_tf_template
from utils.gdf_utils import repair_geometries, show_columns, verify_gdf_columns
from utils.add_common_columns import add_common_columns
from utils.enrich_polygons import enrich_polygons
from utils.keep_fields import keep_fields
from utils.assign_domains import assign_domains
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [4]:
a_reference_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb"
start_year, end_year = 1950, 2025

process = psutil.Process(os.getpid())

In [4]:
from enrich.enrich_Timber_Industry import enrich_Timber_Industry

In [5]:
ti_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\V2.0\Industry_spatial_FFSC_MOU_V2.0\FFSC_MOU_v2_IndustryOnly.gdb"
ti_input_layer_name = "FFSC_MOU_IndustryOnly_Pol"

output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\Timber_Spatial_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"Timber_Industry_Spatial_{datetime.today().strftime('%Y%m%d')}"



In [6]:
enrich_Timber_Industry(ti_input_gdb_path,
                       ti_input_layer_name,
                       a_reference_gdb_path,
                       start_year,
                       end_year,
                       output_gdb_path,
                       output_layer_name)

2025-05-15 16:48:12,646 INFO  [enrich.Timber_Industry]  Load the Timeber Industry Spatial Layer into a GeoDataFrame
2025-05-15 16:48:13,254 INFO  [enrich.Timber_Industry]     time for loading FFSC_MOU_IndustryOnly_Pol: 0.6079411506652832
2025-05-15 16:48:13,255 INFO  [enrich.Timber_Industry]     all required columns are present.
2025-05-15 16:48:13,333 INFO  [enrich.Timber_Industry]  Performing Standardization...
2025-05-15 16:48:13,333 INFO  [enrich.Timber_Industry]     step 1/15 Clip Features to California...
2025-05-15 16:48:56,574 INFO  [enrich.Timber_Industry]        time for loading California and clipping: 43.239997148513794
2025-05-15 16:48:56,574 INFO  [enrich.Timber_Industry]     step 2/15 Repairing Geometry...
2025-05-15 16:48:56,882 INFO  [enrich.Timber_Industry]     step 3/15 Adding Common Columns...
2025-05-15 16:48:56,918 INFO  [enrich.Timber_Industry]     step 4/15 Transfering Values...
2025-05-15 16:48:56,919 INFO  [enrich.Timber_Industry]     step 5/15 Calculating Sta

2025-05-15 16:53:59,671 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-05-15 16:53:59,672 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-05-15 16:53:59,672 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-05-15 16:53:59,675 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-05-15 16:53:59,675 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-05-15 16:53:59,675 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-05-15 16:53:59,676 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category is 'Watershed Improvement'
2025-05-15 16:53:59,676 INFO  [utils.counts_to_mas   ]              counts step 7/8: set to 'NO' if Agency is 'Other' and Admin is 'CARB'


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\Timber_Spatial_1950_2025.gdb


2025-05-15 16:54:00,730 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\Timber_Spatial_1950_2025.gdb Timber_Industry_Spatial_20250515


In [4]:
from multiprocessing.managers import NamespaceProxy, BaseManager
from pandas import DataFrame
import inspect
import os

class MyDataFrame(DataFrame):
    def __getstate__(self):
        print(f'dataframe being pickled in pid {os.getpid()}')
        return super().__getstate__()

    def __setstate__(self, state):
        print(f'dataframe being unpickled in pid {os.getpid()}')
        print()
        return super().__setstate__(state)


class ObjProxy(NamespaceProxy):
    """Returns a proxy instance for any user defined data-type. The proxy instance will have the namespace and
    functions of the data-type (except private/protected callables/attributes). Furthermore, the proxy will be
    pickable and can its state can be shared among different processes. """

    @classmethod
    def populate_obj_attributes(cls, real_cls):
        DISALLOWED = set(dir(cls))
        ALLOWED = ['__sizeof__', '__eq__', '__ne__', '__le__', '__repr__', '__dict__', '__lt__',
                   '__gt__']
        DISALLOWED.add('__class__')
        new_dict = {}
        for (attr, value) in inspect.getmembers(real_cls, callable):
            if attr not in DISALLOWED or attr in ALLOWED:
                new_dict[attr] = proxy_wrap(attr)
        return new_dict


def proxy_wrap(attr):
    """ This method creates function that calls the proxified object's method."""
    def f(self, *args, **kwargs):

        # _callmethod is the method that proxies provided by multiprocessing use to call methods in the proxified object
        return self._callmethod(attr, args, kwargs)

    return f

# Create a class during runtime
new_dict = ObjProxy.populate_obj_attributes(MyDataFrame)
DataFrameObjProxy = type("DataFrameObjProxy", (ObjProxy,), new_dict)
BaseManager.register('DataFrame', MyDataFrame, DataFrameObjProxy, exposed=tuple(dir(DataFrameObjProxy)))

In [4]:
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\USFS_{}_{}.gdb".format(start_year, end_year)
if __name__ == '__main__':
    from enrich.enrich_USFS import enrich_USFS
    #from utils.mp_df import MyDataFrame, ObjProxy
    
    

    
    
    
    
    region_ids = ["05"]#["04", "05", "06"]
    for region_id in region_ids:    
        usfs_input_gdb_path = r'D:\WORK\wildfire\Interagency-Tracking-System\V2.0\FACTS_V2.0\Actv_CommonAttribute_PL_Region{}.gdb'.format(region_id)
        usfs_input_layer_name = "Actv_CommonAttribute_PL"
        output_layer_name = f"USFS_Region{region_id}_enriched_{datetime.today().strftime('%Y%m%d')}"
        # init multiprocessing manager in main module for Windows fork

        
        enrich_USFS(usfs_input_gdb_path,
                    usfs_input_layer_name,
                    a_reference_gdb_path,
                    start_year,
                    end_year,
                    output_gdb_path,
                    output_layer_name,
                    )


2025-06-04 05:25:31,057 INFO  [enrich.enrich_USFS    ]  Loading the USFS data into GeoDataFrames: Actv_CommonAttribute_PL_Region05.gdb : Actv_CommonAttribute_PL
2025-06-04 05:25:31,058 INFO  [enrich.enrich_USFS    ]     Loading USFS data from cache
2025-06-04 05:25:37,050 INFO  [enrich.enrich_USFS    ]        records: 748591
2025-06-04 05:25:37,051 INFO  [enrich.enrich_USFS    ]        time for loading USFS: 5.9931745529174805
2025-06-04 05:25:37,052 INFO  [enrich.enrich_USFS    ]     all required columns are present.
2025-06-04 05:25:48,340 INFO  [enrich.enrich_USFS    ]  Performing Standardization...
2025-06-04 05:25:48,394 INFO  [enrich.enrich_USFS    ]     found 55793 rows with empty geometry
2025-06-04 05:25:48,394 INFO  [enrich.enrich_USFS    ]     drop 55793 rows with empty geometry
2025-06-04 05:25:49,711 INFO  [enrich.enrich_USFS    ]     records in California: 683960
2025-06-04 05:25:49,712 INFO  [enrich.enrich_USFS    ]     step 1/8 Selecting Features...
2025-06-04 05:25:51,

2025-06-04 06:20:36,416 INFO  [utils.enrich_polygons ]                 saved enriched chunks into the cache
2025-06-04 06:20:36,416 INFO  [utils.enrich_polygons ]              ================ processing chunk 5 ================
2025-06-04 06:20:37,091 INFO  [utils.enrich_polygons ]              enrich step 2/32 filter broad veg types using the bounding box
2025-06-04 06:20:37,681 INFO  [utils.enrich_polygons ]                 original broad veg type records: 3736147 
2025-06-04 06:20:37,682 INFO  [utils.enrich_polygons ]                 filtered broad veg type records: 3087932 
2025-06-04 06:20:37,682 INFO  [utils.enrich_polygons ]                 records for summary: 10000
2025-06-04 06:20:37,682 INFO  [utils.enrich_polygons ]              enrich step 3/32 determining broad veg types
2025-06-04 06:20:37,683 INFO  [utils.enrich_polygons ]              enrich step 4/32 joining with broad veg types
2025-06-04 06:20:41,358 INFO  [utils.enrich_polygons ]                 joined records: 36

2025-06-04 07:42:16,393 INFO  [utils.enrich_polygons ]              ================ processing chunk 11 ================
2025-06-04 07:42:16,987 INFO  [utils.enrich_polygons ]              enrich step 2/32 filter broad veg types using the bounding box
2025-06-04 07:42:17,440 INFO  [utils.enrich_polygons ]                 original broad veg type records: 3736147 
2025-06-04 07:42:17,441 INFO  [utils.enrich_polygons ]                 filtered broad veg type records: 1990299 
2025-06-04 07:42:17,441 INFO  [utils.enrich_polygons ]                 records for summary: 10000
2025-06-04 07:42:17,441 INFO  [utils.enrich_polygons ]              enrich step 3/32 determining broad veg types
2025-06-04 07:42:17,442 INFO  [utils.enrich_polygons ]              enrich step 4/32 joining with broad veg types
2025-06-04 07:42:20,097 INFO  [utils.enrich_polygons ]                 joined records: 23832
2025-06-04 07:42:20,098 INFO  [utils.enrich_polygons ]              enrich step 5/32 concurrent calcula

2025-06-04 09:06:37,565 INFO  [utils.enrich_polygons ]              enrich step 2/32 filter broad veg types using the bounding box
2025-06-04 09:06:37,967 INFO  [utils.enrich_polygons ]                 original broad veg type records: 3736147 
2025-06-04 09:06:37,967 INFO  [utils.enrich_polygons ]                 filtered broad veg type records: 2070119 
2025-06-04 09:06:37,967 INFO  [utils.enrich_polygons ]                 records for summary: 10000
2025-06-04 09:06:37,968 INFO  [utils.enrich_polygons ]              enrich step 3/32 determining broad veg types
2025-06-04 09:06:37,968 INFO  [utils.enrich_polygons ]              enrich step 4/32 joining with broad veg types
2025-06-04 09:06:40,234 INFO  [utils.enrich_polygons ]                 joined records: 19140
2025-06-04 09:06:40,235 INFO  [utils.enrich_polygons ]              enrich step 5/32 concurrent calculate veg type for each polygon
2025-06-04 09:06:40,235 INFO  [utils.enrich_polygons ]              enrich step 5/32 MULTIPRO

2025-06-04 10:35:47,704 INFO  [utils.enrich_polygons ]                 original broad veg type records: 3736147 
2025-06-04 10:35:47,704 INFO  [utils.enrich_polygons ]                 filtered broad veg type records: 3291688 
2025-06-04 10:35:47,705 INFO  [utils.enrich_polygons ]                 records for summary: 10000
2025-06-04 10:35:47,705 INFO  [utils.enrich_polygons ]              enrich step 3/32 determining broad veg types
2025-06-04 10:35:47,705 INFO  [utils.enrich_polygons ]              enrich step 4/32 joining with broad veg types
2025-06-04 10:37:38,766 INFO  [utils.enrich_polygons ]                 joined records: 48167
2025-06-04 10:37:38,767 INFO  [utils.enrich_polygons ]              enrich step 5/32 concurrent calculate veg type for each polygon
2025-06-04 10:37:38,767 INFO  [utils.enrich_polygons ]              enrich step 5/32 MULTIPROCESSING DISABLED
2025-06-04 10:52:37,975 INFO  [utils.enrich_polygons ]                 assigning vegetation types is completed
202

...

2025-06-04 13:32:59,946 INFO  [utils.enrich_polygons ]                 records for summary: 10000
2025-06-04 13:32:59,946 INFO  [utils.enrich_polygons ]              enrich step 3/32 determining broad veg types
2025-06-04 13:32:59,947 INFO  [utils.enrich_polygons ]              enrich step 4/32 joining with broad veg types
2025-06-04 13:33:05,327 INFO  [utils.enrich_polygons ]                 joined records: 28579
2025-06-04 13:33:05,329 INFO  [utils.enrich_polygons ]              enrich step 5/32 concurrent calculate veg type for each polygon
2025-06-04 13:33:05,329 INFO  [utils.enrich_polygons ]              enrich step 5/32 MULTIPROCESSING DISABLED
2025-06-04 13:49:32,719 INFO  [utils.enrich_polygons ]                 assigning vegetation types is completed
2025-06-04 13:49:32,881 INFO  [utils.enrich_polygons ]                 saved enriched chunks into the cache
2025-06-04 13:49:32,882 INFO  [utils.enrich_polygons ]              ================ processing chunk 36 ================

2025-06-04 15:09:23,809 INFO  [utils.enrich_polygons ]              enrich step 3/32 determining broad veg types
2025-06-04 15:09:23,809 INFO  [utils.enrich_polygons ]              enrich step 4/32 joining with broad veg types
2025-06-04 15:15:55,344 INFO  [utils.enrich_polygons ]                 joined records: 55433
2025-06-04 15:15:55,345 INFO  [utils.enrich_polygons ]              enrich step 5/32 concurrent calculate veg type for each polygon
2025-06-04 15:15:55,345 INFO  [utils.enrich_polygons ]              enrich step 5/32 MULTIPROCESSING DISABLED
2025-06-04 15:20:49,107 INFO  [utils.enrich_polygons ]                 assigning vegetation types is completed
2025-06-04 15:20:49,287 INFO  [utils.enrich_polygons ]                 saved enriched chunks into the cache
2025-06-04 15:20:50,681 INFO  [utils.enrich_polygons ]                 time for summarizing veg types: 35658.63615942001
2025-06-04 15:20:51,617 INFO  [utils.enrich_polygons ]              enrich step 7/32 select record

2025-06-04 15:27:56,368 INFO  [utils.assign_domains  ]        Created domain 'D_FNDSRC' with 11 values
2025-06-04 15:27:56,374 INFO  [utils.assign_domains  ]        Created domain 'D_AGENCY' with 9 values
2025-06-04 15:27:56,380 INFO  [utils.assign_domains  ]        Created domain 'D_ORGANIZATION' with 32 values
2025-06-04 15:27:56,386 INFO  [utils.assign_domains  ]        Warning: 1 rows with NULL values were dropped from 'D_DATASTATUS'
2025-06-04 15:27:56,387 INFO  [utils.assign_domains  ]        Created domain 'D_DATASTATUS' with 2 values
2025-06-04 15:27:56,393 INFO  [utils.assign_domains  ]        Created domain 'D_DATAMSG' with 3 values
2025-06-04 15:27:56,399 INFO  [utils.assign_domains  ]        Created domain 'D_VERFIEDMSG' with 2 values
2025-06-04 15:27:56,405 INFO  [utils.assign_domains  ]        Created domain 'D_TRMT_GEOM' with 4 values
2025-06-04 15:27:56,405 INFO  [utils.assign_domains  ]        Assign domains to project-related columns
2025-06-04 15:27:56,635 INFO  [uti

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\USFS_1950_2025.gdb


2025-06-04 15:28:00,692 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file
2025-06-04 15:28:28,384 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\USFS_1950_2025.gdb USFS_Region05_enriched_20250604


In [5]:
if __name__ == '__main__':
    region_ids = ["04", "06"]
    for region_id in region_ids:    
        usfs_input_gdb_path = r'D:\WORK\wildfire\Interagency-Tracking-System\V2.0\FACTS_V2.0\Actv_CommonAttribute_PL_Region{}.gdb'.format(region_id)
        usfs_input_layer_name = "Actv_CommonAttribute_PL"
        output_layer_name = f"USFS_Region{region_id}_enriched_{datetime.today().strftime('%Y%m%d')}"
        # init multiprocessing manager in main module for Windows fork

        
        enrich_USFS(usfs_input_gdb_path,
                    usfs_input_layer_name,
                    a_reference_gdb_path,
                    start_year,
                    end_year,
                    output_gdb_path,
                    output_layer_name,
                    )

2025-06-04 16:03:50,899 INFO  [enrich.enrich_USFS    ]  Loading the USFS data into GeoDataFrames: Actv_CommonAttribute_PL_Region04.gdb : Actv_CommonAttribute_PL
2025-06-04 16:03:50,900 INFO  [enrich.enrich_USFS    ]     Loading USFS data from source and cache the data
2025-06-04 16:04:19,996 INFO  [enrich.enrich_USFS    ]        records: 337783
2025-06-04 16:04:19,996 INFO  [enrich.enrich_USFS    ]        time for loading USFS: 29.09639286994934
2025-06-04 16:04:19,997 INFO  [enrich.enrich_USFS    ]     all required columns are present.
2025-06-04 16:04:25,004 INFO  [enrich.enrich_USFS    ]  Performing Standardization...
2025-06-04 16:04:25,138 INFO  [enrich.enrich_USFS    ]     found 137057 rows with empty geometry
2025-06-04 16:04:25,139 INFO  [enrich.enrich_USFS    ]     drop 137057 rows with empty geometry
2025-06-04 16:04:26,229 INFO  [enrich.enrich_USFS    ]     records in California: 979
2025-06-04 16:04:26,230 INFO  [enrich.enrich_USFS    ]     step 1/8 Selecting Features...
20

2025-06-04 16:05:30,635 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-06-04 16:05:30,635 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-06-04 16:05:30,635 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-06-04 16:05:30,636 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-06-04 16:05:30,637 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-06-04 16:05:30,637 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-06-04 16:05:30,638 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category is 'Watershed Improvement'
2025-06-04 16:05:30,638 INFO  [utils.counts_to_mas   ]              counts step 7/8: set to 'NO' if Agency is 'Other' and Admin is 'CARB'


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\USFS_1950_2025.gdb


2025-06-04 16:05:31,401 INFO  [enrich.enrich_USFS    ]  Loading the USFS data into GeoDataFrames: Actv_CommonAttribute_PL_Region06.gdb : Actv_CommonAttribute_PL
2025-06-04 16:05:31,402 INFO  [enrich.enrich_USFS    ]     Loading USFS data from source and cache the data
2025-06-04 16:07:14,442 INFO  [enrich.enrich_USFS    ]        records: 1213138
2025-06-04 16:07:14,442 INFO  [enrich.enrich_USFS    ]        time for loading USFS: 103.04078197479248
2025-06-04 16:07:14,443 INFO  [enrich.enrich_USFS    ]     all required columns are present.
2025-06-04 16:07:39,281 INFO  [enrich.enrich_USFS    ]  Performing Standardization...
2025-06-04 16:07:39,523 INFO  [enrich.enrich_USFS    ]     found 181936 rows with empty geometry
2025-06-04 16:07:39,524 INFO  [enrich.enrich_USFS    ]     drop 181936 rows with empty geometry
2025-06-04 16:07:44,763 INFO  [enrich.enrich_USFS    ]     records in California: 1205
2025-06-04 16:07:44,764 INFO  [enrich.enrich_USFS    ]     step 1/8 Selecting Features...

2025-06-04 16:08:49,362 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-06-04 16:08:49,362 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-06-04 16:08:49,363 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-06-04 16:08:49,364 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-06-04 16:08:49,364 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-06-04 16:08:49,364 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-06-04 16:08:49,365 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category is 'Watershed Improvement'
2025-06-04 16:08:49,365 INFO  [utils.counts_to_mas   ]              counts step 7/8: set to 'NO' if Agency is 'Other' and Admin is 'CARB'


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\USFS_1950_2025.gdb


In [4]:
from enrich.enrich_Timber_Nonspatial import enrich_Timber_Nonspatial

In [5]:


tn_input_excel_path = r'D:\WORK\wildfire\Interagency-Tracking-System\2023\Industry_nonspatial_2023\Timber_Industry_Acres_{}.xlsx'
output_gdb_path =  r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\Timber_Nonspatial_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"Timber_Nonspatial_{datetime.today().strftime('%Y%m%d')}"



In [6]:


tn_input_excel_path = r'D:\WORK\wildfire\Interagency-Tracking-System\V2.0\Industry_nonspatial_V2.0\timber2024concat.xlsx'
output_gdb_path =  r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\Timber_Nonspatial_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"Timber_Nonspatial_{datetime.today().strftime('%Y%m%d')}"


In [7]:
enrich_Timber_Nonspatial(tn_input_excel_path.format('concat'),
                         a_reference_gdb_path,
                         start_year,
                         end_year,
                         output_gdb_path,
                         output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")

2025-05-23 11:57:17,548 INFO  [enrich.Timber_NSpatial]  Load the Timeber Industry Nonspatial data into a DataFrame
2025-05-23 11:57:17,657 INFO  [enrich.Timber_NSpatial]     time for loading D:\WORK\wildfire\Interagency-Tracking-System\V2.0\Industry_nonspatial_V2.0\Timber Industry Acres 2024 for UCSD 13May2025.xlsx: 0.10840058326721191
2025-05-23 11:57:17,657 INFO  [enrich.Timber_NSpatial]  Performing Standardization
2025-05-23 11:57:17,657 INFO  [enrich.Timber_NSpatial]     step 1/10 convert Excel sheet to table
2025-05-23 11:57:17,662 INFO  [enrich.Timber_NSpatial]     all required columns are present.
2025-05-23 11:57:17,663 INFO  [enrich.Timber_NSpatial]     step 2/10 rename and add fields
2025-05-23 11:57:17,663 INFO  [enrich.Timber_NSpatial]     step 3/10 adding common columns...
2025-05-23 11:57:17,684 INFO  [enrich.Timber_NSpatial]     step 4/10 calculate fields
2025-05-23 11:57:17,686 INFO  [enrich.Timber_NSpatial]     step 5/10 converting Table to Geodataframe
2025-05-23 11:5

2025-05-23 11:57:24,277 INFO  [utils.assign_domains  ]        Created domain 'D_VERFIEDMSG' with 2 values
2025-05-23 11:57:24,282 INFO  [utils.assign_domains  ]        Created domain 'D_TRMT_GEOM' with 4 values
2025-05-23 11:57:24,283 INFO  [utils.assign_domains  ]        Assign domains to project-related columns
2025-05-23 11:57:24,287 INFO  [utils.assign_domains  ]        Assign domains to treatment-related columns
2025-05-23 11:57:24,292 INFO  [utils.assign_domains  ]        Assign domains to activity-related columns
2025-05-23 11:57:24,301 INFO  [enrich.Timber_NSpatial]     step 10/10 Save Result...
2025-05-23 11:57:24,301 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-05-23 11:57:24,301 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-05-23 11:57:24,302 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file
2025-05-23 11:57:24,376 INFO  [utils.save_gdf_to_gdb ]        F

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\Timber_Nonspatial_1950_2025.gdb


In [5]:
from enrich.enrich_NPS import enrich_NPS_from_gdb

In [6]:
nps_gdb_path = r'D:\WORK\wildfire\Interagency-Tracking-System\V2.0\NPS_V2.0\NPS_V2.0\NPS_V2_0_20250331.shp'
nps_layer_name = None
output_gdb_path =  r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\NPS_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"NPS_enriched_{datetime.today().strftime('%Y%m%d')}"



In [7]:
enrich_NPS_from_gdb(nps_gdb_path,
                    nps_layer_name,
                    a_reference_gdb_path,
                    start_year,
                    end_year,
                    output_gdb_path,
                    output_layer_name)

2025-07-18 12:13:09,392 INFO  [enrich.enrich_NPS     ]  Load the NPS data into a GeoDataFrame
2025-07-18 12:13:09,808 INFO  [enrich.enrich_NPS     ]     time for loading None: 0.4152565002441406
2025-07-18 12:13:09,809 INFO  [enrich.enrich_NPS     ]     all required columns are present.
2025-07-18 12:13:10,244 INFO  [enrich.enrich_NPS     ]  Performing Standardization...
2025-07-18 12:13:10,244 INFO  [enrich.enrich_NPS     ]     step 1/11 select after 1995
2025-07-18 12:13:10,261 INFO  [enrich.enrich_NPS     ]     step 2/11 repairing geometry
2025-07-18 12:13:13,107 INFO  [enrich.enrich_NPS     ]     step 3/11 clip features by CA
2025-07-18 12:13:23,198 INFO  [enrich.enrich_NPS     ]     step 4/11 dissolve to implement multipart polygons
2025-07-18 12:13:23,723 INFO  [enrich.enrich_NPS     ]     step 5/11 rename and add fields
2025-07-18 12:13:23,747 INFO  [enrich.enrich_NPS     ]     step 6/11 import attributes
2025-07-18 12:13:23,767 INFO  [enrich.enrich_NPS     ]     step 7/11 Remov

2025-07-18 12:14:59,274 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-18 12:14:59,274 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-18 12:14:59,275 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-18 12:14:59,276 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-18 12:14:59,277 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-18 12:14:59,277 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category is 'Watershed Improvement'
2025-07-18 12:14:59,278 INFO  [utils.counts_to_mas   ]              counts step 7/8: set to 'NO' if Agency is 'Other' and Admin is 'CARB'
2025-07-18 12:14:59,279 INFO  [utils.counts_to_mas   ]              counts step 8/8: set to

NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED


2025-07-18 12:14:59,499 INFO  [enrich.enrich_NPS     ]     step 9/11 adding treatment ID
2025-07-18 12:14:59,509 INFO  [enrich.enrich_NPS     ]     step 10/11 Assign Domains...
2025-07-18 12:14:59,762 INFO  [utils.assign_domains  ]        Created domain 'D_OBJECTIVE' with 27 values
2025-07-18 12:14:59,770 INFO  [utils.assign_domains  ]        Created domain 'D_STATUS' with 6 values
2025-07-18 12:14:59,779 INFO  [utils.assign_domains  ]        Created domain 'D_CNTY' with 59 values
2025-07-18 12:14:59,786 INFO  [utils.assign_domains  ]        Created domain 'D_IN_WUI' with 4 values
2025-07-18 12:14:59,794 INFO  [utils.assign_domains  ]        Created domain 'D_ACTVDSCRP' with 61 values
2025-07-18 12:14:59,800 INFO  [utils.assign_domains  ]        Created domain 'D_ACTVCAT' with 8 values
2025-07-18 12:14:59,806 INFO  [utils.assign_domains  ]        Created domain 'D_USERDEFINED' with 2 values
2025-07-18 12:14:59,813 INFO  [utils.assign_domains  ]        Created domain 'D_BVT' with 8 valu

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\NPS_1950_2025.gdb


In [8]:
nps_gdf = gpd.read_file(output_gdb_path, driver='OpenFileGDB', layer=output_layer_name)

In [5]:
from enrich.enrich_BLM import enrich_BLM

In [6]:
blm_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\V2.0\BLM_V2.0\BLM_V2.0\BLM_V2_0_20250331.shp"
blm_input_layer_name = None
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\BLM_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"BLM_enriched_20250710"

In [7]:

enrich_BLM(blm_input_gdb_path,
               blm_input_layer_name,
               a_reference_gdb_path,
               start_year,
               end_year,
               output_gdb_path,
               output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")

2025-07-14 11:56:25,055 INFO  [enrich.enrich_BLM     ]  Load the BLM data into a GeoDataFrame
2025-07-14 11:56:25,146 INFO  [enrich.enrich_BLM     ]     time for loading None: 0.09030699729919434
2025-07-14 11:56:25,146 INFO  [enrich.enrich_BLM     ]     all required columns are present.
2025-07-14 11:56:25,147 INFO  [enrich.enrich_BLM     ]  Performing Standardization...
2025-07-14 11:56:25,147 INFO  [enrich.enrich_BLM     ]     step 1/15 Clip Features to California...
2025-07-14 11:56:38,718 INFO  [enrich.enrich_BLM     ]     step 2/15 Repairing Geometry...
2025-07-14 11:56:39,138 INFO  [enrich.enrich_BLM     ]     step 3/15 Adding Common Columns...
2025-07-14 11:56:39,164 INFO  [enrich.enrich_BLM     ]     step 4/15 Transfering Values...
2025-07-14 11:56:39,165 INFO  [enrich.enrich_BLM     ]     step 5/15 Calculating Start and End Date...
2025-07-14 11:56:39,177 INFO  [enrich.enrich_BLM     ]     step 6/15 Calculating Status...
2025-07-14 11:56:39,178 INFO  [enrich.enrich_BLM     ] 

2025-07-14 11:59:23,611 INFO  [utils.crosswalk       ]              cross step 7/8 standardize domains
2025-07-14 11:59:23,643 INFO  [utils.crosswalk       ]              cross step 8/8 counts towards MAS
2025-07-14 11:59:23,643 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-07-14 11:59:23,643 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-14 11:59:23,644 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-14 11:59:23,645 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-14 11:59:23,646 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-14 11:59:23,646 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-14 11:59:23,646 INFO  [utils.counts_to_mas   ]              counts ste

TBD
TBD
NOT_DEFINED
NOT_DEFINED
FOUND TBD ACTIVITY DESCRIPTION:
TBD


2025-07-14 11:59:23,865 INFO  [enrich.enrich_BLM     ]     step 13/15 Calculate Treatment ID...
2025-07-14 11:59:23,868 INFO  [enrich.enrich_BLM     ]     step 14/15 Assign Domains...
2025-07-14 11:59:24,153 INFO  [utils.assign_domains  ]        Created domain 'D_OBJECTIVE' with 27 values
2025-07-14 11:59:24,159 INFO  [utils.assign_domains  ]        Created domain 'D_STATUS' with 6 values
2025-07-14 11:59:24,166 INFO  [utils.assign_domains  ]        Created domain 'D_CNTY' with 59 values
2025-07-14 11:59:24,171 INFO  [utils.assign_domains  ]        Created domain 'D_IN_WUI' with 4 values
2025-07-14 11:59:24,178 INFO  [utils.assign_domains  ]        Created domain 'D_ACTVDSCRP' with 61 values
2025-07-14 11:59:24,183 INFO  [utils.assign_domains  ]        Created domain 'D_ACTVCAT' with 8 values
2025-07-14 11:59:24,189 INFO  [utils.assign_domains  ]        Created domain 'D_USERDEFINED' with 2 values
2025-07-14 11:59:24,194 INFO  [utils.assign_domains  ]        Created domain 'D_BVT' with

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\BLM_1950_2025.gdb


2025-07-14 11:59:24,516 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\BLM_1950_2025.gdb BLM_enriched_20250710
2025-07-14 11:59:24,527 INFO  [its_logging.logger_config]  Memory usage: 3880.41 MB


In [4]:
from enrich.enrich_CNRA import enrich_CNRA

In [5]:
gpd.list_layers(r"D:\WORK\wildfire\Interagency-Tracking-System\V2.0\CNRA_V2.0\CNRA_TRMT_DATA_Pv6c_20250620.gdb")

,name,geometry_type
0,TREATMENT_POINT,Point
1,TREATMENT_LINE,MultiLineString
2,TREATMENT_POLY,MultiPolygon
3,PROJECT_POLY,MultiPolygon
4,ACTIVITIES,None


In [6]:

cnra_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\V2.0\CNRA_V2.0\CNRA_TRMT_DATA_Pv6c_20250620.gdb"
cnra_polygon_layer_name = "TREATMENT_POLY"
cnra_line_layer_name = "TREATMENT_LINE"
cnra_point_layer_name = "TREATMENT_POINT"
cnra_project_polygon_layer_name = "PROJECT_POLY"
cnra_activity_layer_name = "ACTIVITIES"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CNRA_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"CNRA_enriched_{datetime.today().strftime('%Y%m%d')}"


In [7]:

enrich_CNRA(cnra_input_gdb_path,
            cnra_polygon_layer_name,
            cnra_line_layer_name,
            cnra_point_layer_name,
            cnra_project_polygon_layer_name,
            cnra_activity_layer_name,
            a_reference_gdb_path,
            start_year,
            end_year,
            output_gdb_path,
            output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")


2025-06-26 22:54:47,950 INFO  [enrich.enrich_CNRA    ]  Load the CNRA polygon layer into a GeoDataFrame
2025-06-26 22:54:48,552 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-06-26 22:54:49,456 INFO  [enrich.enrich_CNRA    ]  Load the CNRA line layer into a GeoDataFrame
2025-06-26 22:54:49,477 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-06-26 22:54:49,481 INFO  [enrich.enrich_CNRA    ]  Load the CNRA point layer into a GeoDataFrame
2025-06-26 22:54:49,507 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-06-26 22:54:49,508 INFO  [enrich.enrich_CNRA    ]  Load the CNRA project polygon layer into a GeoDataFrame
2025-06-26 22:54:49,778 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-06-26 22:54:50,335 INFO  [enrich.enrich_CNRA    ]  Load the CNRA activity layer into a DataFrame
2025-06-26 22:54:51,230 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-06-26 22:

!!!!
13092
2702


2025-06-26 22:54:56,224 INFO  [enrich.enrich_CNRA    ]     Part 7 Calculate Board Vegetation Types, Ownership and Others ... 
2025-06-26 22:54:56,257 INFO  [utils.enrich_polygons ]        Executing Polygon Enrichments...
2025-06-26 22:54:56,257 INFO  [utils.enrich_polygons ]           Calculating Broad Vegetation Type...
2025-06-26 22:54:56,258 INFO  [utils.enrich_polygons ]              enrich step 1/32 summarize veg within polygons
2025-06-26 22:55:00,694 INFO  [utils.enrich_polygons ]                 Loaded Broad_Vegetation_Types from cache
2025-06-26 22:55:00,694 INFO  [utils.enrich_polygons ]                    time for loading Broad_Vegetation_Types: 4.436531066894531
2025-06-26 22:55:00,737 INFO  [utils.enrich_polygons ]              create numerical intermediate columns before multiprocessing
2025-06-26 22:55:00,739 INFO  [utils.enrich_polygons ]              enrich step 2/32 filter broad veg types using the bounding box
2025-06-26 22:55:01,328 INFO  [utils.enrich_polygons ]   

FOUND TBD ACTIVITY DESCRIPTION:
FIRE_PREVENTION
FOUND TBD ACTIVITY DESCRIPTION:
FIRE_PREVENTION
FOUND TBD ACTIVITY DESCRIPTION:
FIRE_PREVENTION
TBD
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
PRESCRB_FIRE
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
ROADWAY_CLEARANCE
NOT_DEFINED
INV_SPECIES_CNTRL
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
FUEL_BREAK
NOT_DEFINED
RIPARIAN_RESTOR
TBD
PRESCRB_FIRE
TBD
PRESCRB_FIRE
TBD
PRESCRB_FIRE


2025-06-26 22:56:04,370 INFO  [utils.crosswalk       ]              cross step 7/8 standardize domains


...

2025-06-26 22:56:26,753 INFO  [utils.enrich_points   ]                 time for loading WUI: 0.1646888256072998
2025-06-26 22:56:26,754 INFO  [utils.enrich_points   ]              enrich step 2/16 select records with null WUI
2025-06-26 22:56:26,755 INFO  [utils.enrich_points   ]              enrich step 3/16 select by WUI location
2025-06-26 22:56:26,791 INFO  [utils.enrich_points   ]              enrich step 4/16 calculate WUI yes
2025-06-26 22:56:26,792 INFO  [utils.enrich_points   ]              enrich step 5/16 select remaining null records
2025-06-26 22:56:26,793 INFO  [utils.enrich_points   ]              enrich step 6/16 calculate WUI no
2025-06-26 22:56:26,794 INFO  [utils.enrich_points   ]           Calculating Ownership, Counties, and Regions...
2025-06-26 22:56:26,794 INFO  [utils.enrich_points   ]              enrich step 7/16 loading CALFIRE_Ownership_Update from cache
2025-06-26 22:56:27,184 INFO  [utils.enrich_points   ]                 time for loading CALFIRE_Ownershi

NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
TBD
NOT_DEFINED
TBD
OTHER_FUELS_REDUCTION
TBD
OTHER_FUELS_REDUCTION
TBD
FOREST_STEWARDSHIP
TBD
OTHER_FOREST_MGMT
TBD
OTHER_FOREST_MGMT
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD
FOUND TBD ACTIVITY DESCRIPTION:
TBD


2025-06-26 22:57:01,244 INFO  [enrich.enrich_CNRA    ]     Part 8 Assign Domains...
2025-06-26 22:57:01,313 INFO  [utils.assign_domains  ]        Created domain 'D_OBJECTIVE' with 27 values
2025-06-26 22:57:01,319 INFO  [utils.assign_domains  ]        Created domain 'D_STATUS' with 6 values
2025-06-26 22:57:01,326 INFO  [utils.assign_domains  ]        Created domain 'D_CNTY' with 59 values
2025-06-26 22:57:01,332 INFO  [utils.assign_domains  ]        Created domain 'D_IN_WUI' with 4 values
2025-06-26 22:57:01,339 INFO  [utils.assign_domains  ]        Created domain 'D_ACTVDSCRP' with 61 values
2025-06-26 22:57:01,344 INFO  [utils.assign_domains  ]        Created domain 'D_ACTVCAT' with 8 values
2025-06-26 22:57:01,350 INFO  [utils.assign_domains  ]        Created domain 'D_USERDEFINED' with 2 values
2025-06-26 22:57:01,356 INFO  [utils.assign_domains  ]        Created domain 'D_BVT' with 8 values
2025-06-26 22:57:01,362 INFO  [utils.assign_domains  ]        Created domain 'D_RESIDUEFAT

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CNRA_1950_2025.gdb


2025-06-26 22:57:01,801 INFO  [its_logging.logger_config]  Memory usage: 7864.69 MB


In [4]:
from enrich.enrich_CalTrans import enrich_Caltrans

In [11]:
# temp file path
caltrans_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\2023\CALTRANS_2023\Caltrans_Vegetation_Management_20_23.gdb"
tree_activity_layer_name = "Caltrans_Vegetation_Management_Trees_ActivitiesTable_20_23"
tree_treatment_layer_name = "Caltrans_Vegetation_Management_Trees_Treatments_20_23"
road_activity_layer_name = "Caltrans_Vegetation_Management_RoadsideLandscape_ActivitiesTable_20_23"
road_treatment_layer_name = "Caltrans_Vegetation_Management_RoadsideLandscape_Treatments_20_23"
start_year = 1950
end_year = 2025
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CalTRANS_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"CalTRANS_enriched_{datetime.today().strftime('%Y%m%d')}"

In [7]:

enrich_Caltrans(caltrans_input_gdb_path,
                tree_activity_layer_name,
                tree_treatment_layer_name,
                road_activity_layer_name,
                road_treatment_layer_name,
                a_reference_gdb_path,
                start_year,
                end_year,
                output_gdb_path,
                output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")

2025-05-12 13:23:51,657 INFO  [enrich.enrich_CalTrans]  Load Caltrans road activity layer into a DataFrame
2025-05-12 13:23:54,068 INFO  [enrich.enrich_CalTrans]     all required columns are present.
2025-05-12 13:23:54,069 INFO  [enrich.enrich_CalTrans]  Load Caltrans road treatment layer into a GeoDataFrame
2025-05-12 13:23:54,190 INFO  [enrich.enrich_CalTrans]     all required columns are present.
2025-05-12 13:23:54,496 INFO  [enrich.enrich_CalTrans]  Performing Standardization
2025-05-12 13:23:54,496 INFO  [enrich.enrich_CalTrans]     step 1/10 merge treatments and activities
2025-05-12 13:23:54,696 INFO  [enrich.enrich_CalTrans]        merged_data has 103426 records
2025-05-12 13:23:54,697 INFO  [enrich.enrich_CalTrans]     step 2/10 repair geometries
2025-05-12 13:23:55,102 INFO  [enrich.enrich_CalTrans]     step 3/10 add standard columns
2025-05-12 13:23:55,559 INFO  [enrich.enrich_CalTrans]     step 4/10 calculate column values
2025-05-12 13:23:57,432 INFO  [enrich.enrich_CalT

2025-05-12 13:29:29,234 INFO  [utils.assign_domains  ]        Created domain 'D_FNDSRC' with 11 values
2025-05-12 13:29:29,239 INFO  [utils.assign_domains  ]        Created domain 'D_AGENCY' with 9 values
2025-05-12 13:29:29,246 INFO  [utils.assign_domains  ]        Created domain 'D_ORGANIZATION' with 32 values
2025-05-12 13:29:29,252 INFO  [utils.assign_domains  ]        Warning: 1 rows with NULL values were dropped from 'D_DATASTATUS'
2025-05-12 13:29:29,252 INFO  [utils.assign_domains  ]        Created domain 'D_DATASTATUS' with 2 values
2025-05-12 13:29:29,258 INFO  [utils.assign_domains  ]        Created domain 'D_DATAMSG' with 3 values
2025-05-12 13:29:29,263 INFO  [utils.assign_domains  ]        Created domain 'D_VERFIEDMSG' with 2 values
2025-05-12 13:29:29,269 INFO  [utils.assign_domains  ]        Created domain 'D_TRMT_GEOM' with 4 values
2025-05-12 13:29:29,269 INFO  [utils.assign_domains  ]        Assign domains to project-related columns
2025-05-12 13:29:29,333 INFO  [uti

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CalTRANS_1950_2025.gdb


2025-05-12 13:29:47,765 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CalTRANS_1950_2025.gdb CalTRANS_enriched_20250512
2025-05-12 13:29:47,873 INFO  [its_logging.logger_config]  Memory usage: 4067.48 MB


In [5]:
# temp file path
caltrans_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\V2.0\CALTRANS_V2.0\Caltrans_Vegetation_Management_Treatments_2024.gdb"
tree_activity_layer_name = "Caltrans_Vegetation_Management_Trees_ActivitiesTable_2024"
tree_treatment_layer_name = "Caltrans_Vegetation_Management_Tree_Treatments_2024"
road_activity_layer_name = "Caltrans_Vegetation_Management_RoadsideLandscape_ActivitiesTable_2024"
road_treatment_layer_name = "Caltrans_Vegetation_Management_RoadsideLandscape_Treatments_2024"
start_year = 1950
end_year = 2025
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CalTRANS_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"CalTRANS_enriched_{datetime.today().strftime('%Y%m%d')}"

In [6]:

enrich_Caltrans(caltrans_input_gdb_path,
                tree_activity_layer_name,
                tree_treatment_layer_name,
                road_activity_layer_name,
                road_treatment_layer_name,
                a_reference_gdb_path,
                start_year,
                end_year,
                output_gdb_path,
                output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")

2025-05-12 14:50:48,757 INFO  [enrich.enrich_CalTrans]  Load Caltrans road activity layer into a DataFrame
2025-05-12 14:50:49,409 INFO  [enrich.enrich_CalTrans]     all required columns are present.
2025-05-12 14:50:49,410 INFO  [enrich.enrich_CalTrans]  Load Caltrans road treatment layer into a GeoDataFrame
2025-05-12 14:50:49,441 INFO  [enrich.enrich_CalTrans]     all required columns are present.
2025-05-12 14:50:49,516 INFO  [enrich.enrich_CalTrans]  Performing Standardization
2025-05-12 14:50:49,516 INFO  [enrich.enrich_CalTrans]     step 1/10 merge treatments and activities
2025-05-12 14:50:49,568 INFO  [enrich.enrich_CalTrans]        merged_data has 27843 records
2025-05-12 14:50:49,569 INFO  [enrich.enrich_CalTrans]     step 2/10 repair geometries
2025-05-12 14:50:49,677 INFO  [enrich.enrich_CalTrans]     step 3/10 add standard columns
2025-05-12 14:50:49,820 INFO  [enrich.enrich_CalTrans]     step 4/10 calculate column values
2025-05-12 14:50:50,739 INFO  [enrich.enrich_CalTr

2025-05-12 14:53:06,527 INFO  [utils.assign_domains  ]        Created domain 'D_FNDSRC' with 11 values
2025-05-12 14:53:06,533 INFO  [utils.assign_domains  ]        Created domain 'D_AGENCY' with 9 values
2025-05-12 14:53:06,539 INFO  [utils.assign_domains  ]        Created domain 'D_ORGANIZATION' with 32 values
2025-05-12 14:53:06,545 INFO  [utils.assign_domains  ]        Warning: 1 rows with NULL values were dropped from 'D_DATASTATUS'
2025-05-12 14:53:06,545 INFO  [utils.assign_domains  ]        Created domain 'D_DATASTATUS' with 2 values
2025-05-12 14:53:06,551 INFO  [utils.assign_domains  ]        Created domain 'D_DATAMSG' with 3 values
2025-05-12 14:53:06,556 INFO  [utils.assign_domains  ]        Created domain 'D_VERFIEDMSG' with 2 values
2025-05-12 14:53:06,562 INFO  [utils.assign_domains  ]        Created domain 'D_TRMT_GEOM' with 4 values
2025-05-12 14:53:06,562 INFO  [utils.assign_domains  ]        Assign domains to project-related columns
2025-05-12 14:53:06,585 INFO  [uti

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CalTRANS_1950_2025.gdb


2025-05-12 14:53:11,553 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CalTRANS_1950_2025.gdb CalTRANS_enriched_20250512
2025-05-12 14:53:11,582 INFO  [its_logging.logger_config]  Memory usage: 3912.47 MB


In [ ]:
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CalTRANS_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"CalTRANS_enriched_{datetime.today().strftime('%Y%m%d')}"

In [7]:
caltrans_2023 = gpd.read_file(r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CalTRANS_{}_{}.gdb".format(start_year, end_year),
                              driver='OpenFileGDB',
                              layer='CalTRANS_enriched_20250512')

In [8]:
caltrans_2024 = gpd.read_file(r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CalTRANS_{}_{}.gdb".format(start_year, end_year),
                              driver='OpenFileGDB',
                              layer='CalTRANS_enriched_20250512')

In [13]:
caltrans_out = pd.concat([caltrans_2023, caltrans_2024])

In [ ]:
caltrans_out.to_file(r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CalTRANS_{}_{}.gdb".format(start_year, end_year),
                      driver='OpenFileGDB',
                      layer='CalTRANS_enriched_20250512')

In [3]:
report_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\reports.gdb"

In [4]:
gpd.list_layers(report_path)

,name,geometry_type
0,footprint2021,MultiPolygon Z
1,footprint2022,MultiPolygon Z
2,footprint2023,MultiPolygon Z


In [5]:
footprint2023 = gpd.read_file(report_path,driver='OpenFileGDB',layer='footprint2023')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [9]:
footprint2023.groupby(['PRIMARY_OWNERSHIP_GROUP']).ACTIVITY_QUANTITY.sum()

PRIMARY_OWNERSHIP_GROUP
FEDERAL                 260083.279806
LOCAL                     4945.251612
NGO                       2446.897781
PRIVATE_INDUSTRY        224994.549016
PRIVATE_NON-INDUSTRY     59268.893918
STATE                   137141.116163
TRIBAL                    3599.720000
Name: ACTIVITY_QUANTITY, dtype: float64

In [10]:
footprint2023

,TRMTID_USER,ACTIVITY_QUANTITY,AGENCY,PRIMARY_OWNERSHIP_GROUP,COUNTY,BROAD_VEGETATION_TYPE,REGION,WUI,geometry
0,01-DN-101-DN-NOR-NON,32.400000,CALSTA,STATE,DN,Forest,North Coast,Non-WUI,"MULTIPOLYGON (((-342583.596 406960.852, -34258..."
1,01-DN-101-DN-NOR-WUI,32.400000,CALSTA,STATE,DN,Forest,North Coast,Non-WUI,"MULTIPOLYGON (((-342583.596 406960.852, -34258..."
2,01-DN-169-DN-NOR-NON,8.500000,CALSTA,STATE,DN,Urban,North Coast,WUI,"MULTIPOLYGON (((-337005.425 396726.106, -33698..."
3,01-DN-169-DN-NOR-WUI,8.500000,CALSTA,STATE,DN,Urban,North Coast,WUI,"MULTIPOLYGON (((-337005.425 396726.106, -33698..."
4,01-DN-197-DN-NOR-WUI,5.860000,CALSTA,STATE,DN,Forest,North Coast,WUI,"MULTIPOLYGON (((-339603.722 428203.383, -33960..."
...,...,...,...,...,...,...,...,...,...
5105,TI-55,12200.547236,TIMBER,PRIVATE_INDUSTRY,None,FOREST,None,Non-WUI,"MULTIPOLYGON (((-308845.263 -48952.018, -30886..."
5106,TI-58,1525.818568,TIMBER,PRIVATE_INDUSTRY,None,FOREST,None,Non-WUI,"MULTIPOLYGON (((-291682.027 -45293.631, -29168..."
5107,TI-61,30646.452164,TIMBER,PRIVATE_INDUSTRY,None,FOREST,None,Non-WUI,"MULTIPOLYGON (((-286089.472 -38332.964, -28611..."
5108,TI-64,2298.548819,TIMBER,PRIVATE_INDUSTRY,None,FOREST,None,Non-WUI,"MULTIPOLYGON (((-295831.21 -46396.505, -295839..."
